## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,albany,US,42.60,-73.97,35.62,52,42,1.99,scattered clouds
1,1,ponta do sol,PT,32.67,-17.10,67.01,72,48,11.81,scattered clouds
2,2,punta arenas,CL,-53.15,-70.92,51.12,50,57,26.22,broken clouds
3,3,kloulklubed,PW,7.04,134.26,78.80,82,100,16.91,moderate rain
4,4,qaanaaq,GL,77.48,-69.36,-18.27,73,100,11.88,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp_input =float(input("Enter minimum temperature you would like for your vacation\n"))
max_temp_input =float(input("Enter maximum temperature you would like for your vacation\n"))

Enter minimum temperature you would like for your vacation
60
Enter maximum temperature you would like for your vacation
82


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp_input) &
                                        (city_data_df["Max Temp"] >= min_temp_input)]
filtered_cities_df.head()


,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,ponta do sol,PT,32.67,-17.10,67.01,72,48,11.81,scattered clouds
3,3,kloulklubed,PW,7.04,134.26,78.80,82,100,16.91,moderate rain
6,6,bluff,NZ,-46.60,168.33,73.80,71,100,13.18,overcast clouds
7,7,busselton,AU,-33.65,115.33,61.05,67,21,18.28,few clouds
8,8,bubaque,GW,11.28,-15.83,78.64,79,4,14.81,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
filtered_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 311 entries, 1 to 711
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           311 non-null    int64  
 1   City                 311 non-null    object 
 2   Country              308 non-null    object 
 3   Lat                  311 non-null    float64
 4   Lng                  311 non-null    float64
 5   Max Temp             311 non-null    float64
 6   Humidity             311 non-null    int64  
 7   Cloudiness           311 non-null    int64  
 8   Wind Speed           311 non-null    float64
 9   Current Description  311 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 26.7+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = filtered_cities_df.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 308 entries, 1 to 711
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           308 non-null    int64  
 1   City                 308 non-null    object 
 2   Country              308 non-null    object 
 3   Lat                  308 non-null    float64
 4   Lng                  308 non-null    float64
 5   Max Temp             308 non-null    float64
 6   Humidity             308 non-null    int64  
 7   Cloudiness           308 non-null    int64  
 8   Wind Speed           308 non-null    float64
 9   Current Description  308 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 26.5+ KB


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,ponta do sol,PT,67.01,scattered clouds,32.67,-17.10,
3,kloulklubed,PW,78.80,moderate rain,7.04,134.26,
6,bluff,NZ,73.80,overcast clouds,-46.60,168.33,
7,busselton,AU,61.05,few clouds,-33.65,115.33,
8,bubaque,GW,78.64,clear sky,11.28,-15.83,
9,caravelas,BR,80.01,broken clouds,-17.71,-39.25,
11,ambilobe,MG,73.65,moderate rain,-13.20,49.05,
13,port hedland,AU,80.65,overcast clouds,-20.32,118.57,
18,mana,GF,79.77,overcast clouds,5.66,-53.78,
19,nouadhibou,MR,67.03,overcast clouds,20.93,-17.03,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
        
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
        
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel Not Found...skipping")

Hotel Not Found...skipping
Hotel Not Found...skipping
Hotel Not Found...skipping
Hotel Not Found...skipping
Hotel Not Found...skipping
Hotel Not Found...skipping
Hotel Not Found...skipping
Hotel Not Found...skipping
Hotel Not Found...skipping
Hotel Not Found...skipping
Hotel Not Found...skipping
Hotel Not Found...skipping
Hotel Not Found...skipping
Hotel Not Found...skipping
Hotel Not Found...skipping
Hotel Not Found...skipping
Hotel Not Found...skipping
Hotel Not Found...skipping
Hotel Not Found...skipping
Hotel Not Found...skipping


In [11]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,ponta do sol,PT,67.01,scattered clouds,32.67,-17.10,Hotel do Campo
3,kloulklubed,PW,78.80,moderate rain,7.04,134.26,Storyboard Beach Resort
6,bluff,NZ,73.80,overcast clouds,-46.60,168.33,Bluff Homestead - Guesthouse & Campervan Park
7,busselton,AU,61.05,few clouds,-33.65,115.33,Observatory Guest House
8,bubaque,GW,78.64,clear sky,11.28,-15.83,"Casa Dora, Bubaque"


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,ponta do sol,PT,67.01,scattered clouds,32.67,-17.10,Hotel do Campo
3,kloulklubed,PW,78.80,moderate rain,7.04,134.26,Storyboard Beach Resort
6,bluff,NZ,73.80,overcast clouds,-46.60,168.33,Bluff Homestead - Guesthouse & Campervan Park
7,busselton,AU,61.05,few clouds,-33.65,115.33,Observatory Guest House
8,bubaque,GW,78.64,clear sky,11.28,-15.83,"Casa Dora, Bubaque"
...,...,...,...,...,...,...,...
697,kiama,AU,69.39,overcast clouds,-34.68,150.87,The Sebel Kiama Harbourside
698,san carlos de bariloche,AR,77.45,clear sky,-41.15,-71.31,Hotel Patagonia
702,sao felix do xingu,BR,78.13,overcast clouds,-6.64,-51.99,Apart Hotel Rio Xingu
710,temuco,CL,78.57,clear sky,-38.73,-72.60,Hotel RP


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<d1>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp}</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))